<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/EY2024/C7_parallel_function_calling_with_Azure_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Parallel Function Calls and Multiple Function Responses in Gemini

## Overview

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini Pro and Gemini Pro Vision models.

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets you create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

In this tutorial, you'll learn how to work with parallel function calling within Gemini Function Calling, including:
    
- Handling parallel function calls for repeated functions
- Working with parallel function calls across multiple functions
- Extracting multiple function calls from a Gemini response
- Calling multiple functions and returning them to Gemini

### What is parallel function calling?

In previous versions of the Gemini Pro models (prior to May 2024), Gemini would return two or more chained function calls if the model determined that more than one function call was needed before returning a natural language summary. Here, a chained function call means that you get the first function call response, return the API data to Gemini, get a second function call response, return the API data to Gemini, and so on.

In recent versions of specific Gemini Pro models (from May 2024 and on), Gemini has the ability to return two or more function calls in parallel (i.e., two or more function call responses within the first function call response object). Parallel function calling allows you to fan out and parallelize your API calls or other actions that you perform in your application code, so you don't have to work through each function call response and return one-by-one! Refer to the [Gemini Function Calling documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling) for more information on which Gemini model versions support parallel function calling.


<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

In [1]:
!pip install openai wikipedia --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [2]:
api_key = "62d77bb382974118aec75d84d274bb72"
api_version = "2024-02-01" # "2023-05-15"
azure_endpoint = "https://eygpt24.openai.azure.com/"
deployment_name = "gpt-4o"

In [3]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key = api_key,)

### Define helper function

In [4]:
def extract_tool_calls(response: dict):
    tool_calls = []
    tool_call = response.choices[0].message.tool_calls
    if tool_call:
        for tool in tool_call:
            function_name = tool.function.name
            function_args = eval(tool.function.arguments)  # Safely convert string to dictionary
            function_call_dict = {function_name: function_args}
            function_call_dict['id'] = tool.id
            tool_calls.append(function_call_dict)

    return tool_calls

## Example: Parallel function calls on the same function

A great use case for parallel function calling is when you have a function that only accepts one parameter per API call and you need to make repeated calls to that function.

With Parallel Function Calling, rather than having to send N number of API requests to LLM for N number function calls, instead you can send a single API request to LLM, receive N number of Function Call Responses within a single response, make N number of external API calls in your code, then return all of the API responses to LLM in bulk. And you can do all of this without any extra configuration in your function declarations, tools, or requests to LLM.

In this example, you'll do exactly that and use Parallel Function Calling in LLM to ask about multiple topics on [Wikipedia](https://www.wikipedia.org/). Let's get started!


In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    }
]

In [6]:
### Send prompt to OpenAI GPT-4o with function calling
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"

In [7]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice="auto",
)


In [8]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_TPg6ABl3zqJQMA7HMKVESV4K', function=Function(arguments='{"query": "solar panels"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_cGfCcd1EPr1vEanUESUn5QN7', function=Function(arguments='{"query": "renewable energy"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_ELni6fchwPvjRi31GZEDadum', function=Function(arguments='{"query": "battery storage"}', name='search_wikipedia'), type='function')]

In [23]:
response2 = client.chat.completions.create(model='gpt-4o',
                                           messages=[{"role":"user","content":"Define Artificial Intelligence"}],
                                           max_tokens=10,n=2)

print(response2.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8MmbttGEXDtbmUgD6fcyUnFQ5U0p",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Artificial Intelligence (AI) refers to the field of",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected": false
        },
        "protected_material_text": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    },
    {
      "finish_r

In [9]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8MebNy7sWWPtmi9hzBuajutYL0Vn",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_TPg6ABl3zqJQMA7HMKVESV4K",
            "function": {
              "arguments": "{\"query\": \"solar panels\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_cGfCcd1EPr1vEanUESUn5QN7",
            "function": {
              "arguments": "{\"query\": \"renewable energy\"}",
              "name": "search_wikipedia"
            },
            "type": "function"
          },
          {
            "id": "call_ELni6fchwPvjRi31GZEDadum",
            "function": {
              "arguments": "{\"query\": \"battery storage\"}",
              "name": "search_wikipedia"


In [10]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_TPg6ABl3zqJQMA7HMKVESV4K'}, {'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_cGfCcd1EPr1vEanUESUn5QN7'}, {'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_ELni6fchwPvjRi31GZEDadum'}]


In [ ]:
!pip install wikipedia --quiet

In [11]:
### Make external API calls
import wikipedia
api_response = []

# Loop over multiple function calls
for tool in tool_calls:
    print(tool)

    # Make external API call
    result = wikipedia.summary(tool["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'solar panels'}, 'id': 'call_TPg6ABl3zqJQMA7HMKVESV4K'}
{'search_wikipedia': {'query': 'renewable energy'}, 'id': 'call_cGfCcd1EPr1vEanUESUn5QN7'}
{'search_wikipedia': {'query': 'battery storage'}, 'id': 'call_ELni6fchwPvjRi31GZEDadum'}


In [12]:
for res in api_response:
    print(res)

A solar panel is a device that converts sunlight into electricity by using photovoltaic (PV) cells. PV cells are made of materials that produce excited electrons when exposed to light. The electrons flow through a circuit and produce direct current (DC) electricity, which can be used to power various devices or be stored in batteries. Solar panels are also known as solar cell panels, solar electric panels, or PV modules.
Solar panels are usually arranged in groups called arrays or systems. A photovoltaic system consists of one or more solar panels, an inverter that converts DC electricity to alternating current (AC) electricity, and sometimes other components such as controllers, meters, and trackers. Most panels are in solar farms, which  supply the electricity grid as can some rooftop solar.
Some advantages of solar panels are that they use a renewable and clean source of energy, reduce greenhouse gas emissions, and lower electricity bills. Some disadvantages are that they depend on 

In [13]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": 'search_wikipedia',
                   "content": res,})


In [14]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages)

In [15]:

# Display the final summary
print(new_response.choices[0].message.content)

Here's a summary of the findings about solar panels, renewable energy, and battery storage:

### Solar Panels
- **Functionality**: Solar panels convert sunlight into electricity using photovoltaic (PV) cells, which generate direct current (DC) electricity when exposed to light. This can then be converted to alternating current (AC) using an inverter.
- **Components**: A typical photovoltaic system includes solar panels, an inverter, and sometimes other devices like controllers and meters.
- **Applications**: They are used widely in residential, commercial, and industrial settings, and even in space missions.
- **Advantages**: Solar panels use a renewable and clean energy source, reduce greenhouse gas emissions, and can lower electricity costs over time.
- **Disadvantages**: They are dependent on sunlight, require regular maintenance, and have high initial setup costs.

### Renewable Energy
- **Sources**: Includes solar energy, wind power, hydropower, bioenergy, and geothermal energy. S

## Example: Parallel function calls across multiple functions

Another good fit for parallel function calling is when you have multiple, independent functions that you want to be able to call in parallel, which reduces the number of consecutive OpenAI API calls that you need to make and (ideally) reduces the overall response time to the end user who is waiting for a natural language response.

In this example, you'll use Parallel Function Calling in LLM to ask about multiple aspects of topics and articles on [Wikipedia](https://www.wikipedia.org/).


In [24]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "suggest_wikipedia",
            "description": "Get suggested titles from Wikipedia based on the given query term",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for suggested titles on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "summarize_wikipedia",
            "description": "Retrieve brief summaries of Wikipedia articles related to a given topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "Topic to search for article summaries on Wikipedia"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    }
]


In [25]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

In [26]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [27]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_An1DKMDeJ5L1tfCJRBWCFPxO', function=Function(arguments='{"query": "solar system"}', name='search_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_Pj2pn9eNsGL1Sl5YNs9T8h3G', function=Function(arguments='{"query": "solar system"}', name='suggest_wikipedia'), type='function'),
 ChatCompletionMessageToolCall(id='call_O1GprppT2jkdHdQvuxVa57nG', function=Function(arguments='{"topic": "solar system"}', name='summarize_wikipedia'), type='function')]

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that LLM responded with:

In [28]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_An1DKMDeJ5L1tfCJRBWCFPxO'}, {'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_Pj2pn9eNsGL1Sl5YNs9T8h3G'}, {'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_O1GprppT2jkdHdQvuxVa57nG'}]


### Make external API calls

Next, you'll loop through the Function Calls and use the `wikipedia` Python package to make APIs calls and gather information from Wikipedia:

In [29]:
api_response = {}

# Loop over multiple function calls
for tool in tool_calls:
    # Extract the function name
    print(tool)
    function_name = list(tool.keys())[0]

    # Determine which external API call to make
    if function_name == "search_wikipedia":
        result = wikipedia.search(tool["search_wikipedia"]["query"])
    if function_name == "suggest_wikipedia":
        result = wikipedia.suggest(tool["suggest_wikipedia"]["query"])
    if function_name == "summarize_wikipedia":
        result = wikipedia.summary(
            tool["summarize_wikipedia"]["topic"], auto_suggest=False
        )

    # Collect all API responses
    api_response[function_name] = result

{'search_wikipedia': {'query': 'solar system'}, 'id': 'call_An1DKMDeJ5L1tfCJRBWCFPxO'}
{'suggest_wikipedia': {'query': 'solar system'}, 'id': 'call_Pj2pn9eNsGL1Sl5YNs9T8h3G'}
{'summarize_wikipedia': {'topic': 'solar system'}, 'id': 'call_O1GprppT2jkdHdQvuxVa57nG'}


In [30]:
api_response

{'search_wikipedia': ['Solar System',
  'Formation and evolution of the Solar System',
  'List of Solar System objects by size',
  'Solar System (disambiguation)',
  'List of Solar System objects',
  'Passive solar building design',
  'Photovoltaic system',
  'Solar System belts',
  'List of gravitationally rounded objects of the Solar System',
  'Exoplanet'],
 'suggest_wikipedia': 'soler system',
 'summarize_wikipedia': "The Solar System is the gravitationally bound system of the Sun and the objects that orbit it. It was formed about 4.6 billion years ago when a dense region of a molecular cloud collapsed, forming the Sun and a protoplanetary disc. The Sun is a typical star that maintains a balanced equilibrium by the fusion of hydrogen into helium at its core, releasing this energy from its outer photosphere. Astronomers classify it as a G-type main-sequence star.\nThe largest objects that orbit the Sun are the eight planets. In order from the Sun, they are four terrestrial planets (

### Get a natural language summary

Now you can return all of the API responses to LLM so that it can generate a natural language summary:

In [31]:

### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  function_name = list(tool.keys())[0]
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": function_name,
                   "content": res,})


In [32]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages)

In [33]:
# Display the final summary
print(new_response.choices[0].message.content)

### Search Results:
Here are some of the top Wikipedia articles related to the solar system:

1. **Solar System**
   - **Summary**: An extensive overview of the solar system, including its formation, structure, and various components like the planets, moons, and other celestial bodies.

2. **Planets of the Solar System**
   - **Summary**: Detailed information about each planet in the solar system, their characteristics, orbits, and atmospheres.

3. **Formation and Evolution of the Solar System**
   - **Summary**: Discusses the scientific theories and evidence related to the origin and development of the solar system over billions of years.

4. **Moons of the Solar System**
   - **Summary**: Describes the natural satellites orbiting the planets, including their discovery, physical characteristics, and importance.

5. **Asteroids and Comets in the Solar System**
   - **Summary**: Information about the smaller bodies in the solar system, their composition, orbits, and impact on Earth.

##

# Sequential Multi function call

In [36]:
import requests
import json

In [37]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"

    try:
      # Send the API request
      response = requests.get(url)
    except:
      return "Error occurred because location does not exist"

    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }

    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [38]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_wikipedia",
            "description": "Search for articles or any other relevant information on Wikipedia'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Query to search for on Wikipedia"
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    },
    {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },
]

In [59]:
prompt = "Get the name of capital city of Tripura using wikipedia and then tell me the weather information for the same using wikipedia results. "

In [60]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [61]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_p1mJohSkU5CXbXmhdqiHbQIc', function=Function(arguments='{"query":"capital city of Tripura"}', name='search_wikipedia'), type='function')]

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that LLM responded with:

In [62]:
# Extract function calls from the response
tool_calls = extract_tool_calls(response)
print(tool_calls)

[{'search_wikipedia': {'query': 'capital city of Tripura'}, 'id': 'call_p1mJohSkU5CXbXmhdqiHbQIc'}]


In [63]:
### Make external API calls
import wikipedia
api_response = []

# Loop over multiple function calls
for tool in tool_calls:
    print(tool)

    # Make external API call
    result = wikipedia.summary(tool["search_wikipedia"]["query"])

    # Collect all API responses
    api_response.append(result)

{'search_wikipedia': {'query': 'capital city of Tripura'}, 'id': 'call_p1mJohSkU5CXbXmhdqiHbQIc'}


In [64]:
for res in api_response:
    print(res)

Agartala (, Bengali: [agorot̪ɔla] , Kok Borok: [agaratala]) is the capital and the largest city of the Indian state of Tripura, situated on the banks of Haora/Saidra River, about 2 kilometres (1.2 mi) east of the border with Bangladesh and about 2,499 km (1,552 mi) from the national capital, New Delhi. According to 2022 AMC data, Agartala is the second most populous city after Guwahati in Northeast India. It is India's third international internet gateway and being developed under the Smart Cities Mission.




In [65]:
### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id": tool['id'],
                   "role": "tool",
                   "name": 'search_wikipedia',
                   "content": res,})


In [66]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages,
    tools=tools,
    tool_choice='auto')

In [67]:
# Display the final summary
print(new_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8NGRNehtKPy114cKGrY6xrgb1Pdf",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_QBq65Jtj6gbHqZbAM38e2Fey",
            "function": {
              "arguments": "{\"location\":\"Agartala, Tripura\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1726559415,
  "model": "gpt-4o-2024-05-13",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_80a1bad4c7",
  "usage": {
    "completion_tokens": 25,
    "prompt_tokens": 301,
    "total_tokens": 326,
    "completion_tokens_details": null
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
  

In [ ]:
for i in range(10):
  response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto')

  tool_call_flag = response.choices[0].message.tool_calls

  if tool_call:
    response = response.choices[0].message
    tool_calls = response.tool_calls
    for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                print(function_response)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

  else:
    return response.choices[0].message.content



In [73]:
tools = [
    {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },
]

In [90]:
prompt = "what is 2+2? and tell me the weather of bangalore. Answer questions together at the end."

In [91]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    tools=tools,
    tool_choice='auto'
)


In [92]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_lfTIZcEXKhJRzjIF5srNpGKH', function=Function(arguments='{"location": "Bangalore"}', name='get_current_weather'), type='function')]

In [93]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8NT540tyItXaWXNw9dALXn1cAoXU",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_lfTIZcEXKhJRzjIF5srNpGKH",
            "function": {
              "arguments": "{\"location\": \"Bangalore\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1726560199,
  "model": "gpt-4o-2024-05-13",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_80a1bad4c7",
  "usage": {
    "completion_tokens": 32,
    "prompt_tokens": 106,
    "total_tokens": 138,
    "completion_tokens_details": null
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results"

In [97]:
### Generate a natural language summary
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},]

messages.append(response.choices[0].message)
for tool,res in zip(tool_calls,api_response):
  messages.append({"tool_call_id":"call_lfTIZcEXKhJRzjIF5srNpGKH" ,
                   "role": "tool",
                   "name": 'get_current_weather',
                   "content": '{"temperature":"340","unit":"kelvin"}',})


In [98]:
# Now we return all API responses back to GPT-4 to summarize

new_response = client.chat.completions.create(
    model="gpt-4o",
    messages = messages,
    tools=tools,
    tool_choice='auto')

In [99]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'what is 2+2? and tell me the weather of bangalore. Answer questions together at the end.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lfTIZcEXKhJRzjIF5srNpGKH', function=Function(arguments='{"location": "Bangalore"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_lfTIZcEXKhJRzjIF5srNpGKH',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"temperature":"340","unit":"kelvin"}'}]

In [100]:
# Display the final summary
print(new_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-A8NTbq5K9gylL2fMVglMVefn6K2aj",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "1. The answer to 2+2 is 4.\n2. The current weather in Bangalore is 340 K (Kelvin).",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1726560231,
  "model": "gpt-4o-2024-05-13",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_80a1bad4c7",
  "usage": {
    "completion_tokens": 29,
   